In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.43.0 --progress-bar off
!pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install -qqq einops==0.6.1 --progress-bar off

In [ ]:
#Restart kernel after this
!pip install sentencepiece

In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
data = load_dataset("json", data_files="/content/converted_data.json")
data

In [ ]:
#MODEL_NAME = "meta-llama/Llama-2-13b-chat-hf"
MODEL_NAME = "huggyllama/llama-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
#Install it if there is an issue
#!pip install --upgrade protobuf

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [5]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
#print_trainable_parameters(model)

In [6]:
prompt = f"""
<human>: Are Gods Real?
<assistant>:
""".strip()
print(prompt)

<human>: Are Gods Real?
<assistant>:


In [7]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id
generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_new_tokens": 200,
  "pad_token_id": 2,
  "temperature": 0.7,
  "top_p": 0.7,
  "transformers_version": "4.30.0.dev0"
}

In [8]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>: Are Gods Real?
<assistant>: Are Gods Real?
<human>: I don't know.
<assistant>: I don't know either.
<human>: I don't know. I don't know.
<assistant>: I don't know. I don't know.
<human>: I don't know. I don't know. I don't know.
<assistant>: I don't know. I don't know. I don't know.
<human>: I don't know. I don't know. I don't know. I don't know.
<assistant>: I don't know. I don't know. I don't know. I don't know.
<human>: I don't know. I don't know. I don't know. I don't know. I don't know.
<assistant
CPU times: user 23.8 s, sys: 462 ms, total: 24.3 s
Wall time: 27.1 s


In [ ]:
def generate_prompt(data_point):
    return f"""
<human>: {data_point["question"]}
<assistant>: {data_point["answer"]}
""".strip()


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle().map(generate_and_tokenize_prompt)
data

In [10]:
!pip install tensorboardX

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_shell.py:99 in system                      │
│                                                                                                  │
│    96 │   if pip_warn:                                                                           │
│    97 │     kwargs.update({'also_return_output': True})                                          │
│    98 │                                                                                          │
│ ❱  99 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│   100 │                                                                                          │
│   101 │   if pip_warn:                                                                           │
│   102 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py:453 in _system_compat   │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_output=False                          │
│   455   )                                                                                        │
│   456   shell.user_ns['_exit_code'] = result.returncode                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py:167 in _run_command     │
│                                                                                                  │
│   164   """Calls the shell command, forwarding input received on the stdin_socket."""            │
│   165   locale_encoding = locale.getpreferredencoding()                                          │
│   166   if locale_encoding != _ENCODING:                                                         │
│ ❱ 167 │   raise NotImplementedError(                                                             │
│   168 │   │   'A UTF-8 locale is required. Got {}'.format(locale_encoding)                       │
│   169 │   )                                                                                      │
│   170                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [11]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

In [12]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.366500
2,2.038200
3,2.237200
4,2.105600
5,2.298300
6,2.169400
7,2.494600
8,2.009400
9,1.655900
10,1.849700


TrainOutput(global_step=80, training_loss=1.3904458105564117, metrics={'train_runtime': 136.7081, 'train_samples_per_second': 2.341, 'train_steps_per_second': 0.585, 'total_flos': 809216175857664.0, 'train_loss': 1.3904458105564117, 'epoch': 0.46})

In [13]:
trainer.model.save_pretrained("checkSave1")

In [17]:
!pip install huggingface_hub

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_shell.py:99 in system                      │
│                                                                                                  │
│    96 │   if pip_warn:                                                                           │
│    97 │     kwargs.update({'also_return_output': True})                                          │
│    98 │                                                                                          │
│ ❱  99 │   output = _system_commands._system_compat(self, *args, **kwargs)  # pylint:disable=pr   │
│   100 │                                                                                          │
│   101 │   if pip_warn:                                                                           │
│   102 │     _pip.print_previous_import_warning(output)                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py:453 in _system_compat   │
│                                                                                                  │
│   450   # We set a higher depth than the IPython system command since a shell object             │
│   451   # is expected to call this function, thus adding one level of nesting to the             │
│   452   # stack.                                                                                 │
│ ❱ 453   result = _run_command(                                                                   │
│   454 │     shell.var_expand(cmd, depth=2), clear_streamed_output=False                          │
│   455   )                                                                                        │
│   456   shell.user_ns['_exit_code'] = result.returncode                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/google/colab/_system_commands.py:167 in _run_command     │
│                                                                                                  │
│   164   """Calls the shell command, forwarding input received on the stdin_socket."""            │
│   165   locale_encoding = locale.getpreferredencoding()                                          │
│   166   if locale_encoding != _ENCODING:                                                         │
│ ❱ 167 │   raise NotImplementedError(                                                             │
│   168 │   │   'A UTF-8 locale is required. Got {}'.format(locale_encoding)                       │
│   169 │   )                                                                                      │
│   170                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
trainer.model.push_to_hub

In [14]:
generation_config = model.generation_config
generation_config.max_new_tokens = 100
generation_config.temperature = 0
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [15]:
DEVICE = "cuda:0"

prompt = f"""
<human>: What are comets mostly made of??
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


<human>: What are comets mostly made of??
<assistant>: Well, talk about cosmic clutter!Comets are mostly made of ice and dust. They are small, icy bodies that orbit the Sun in the outer reaches of the solar system. They are believed to have originated from the formation of the solar system and are thought to contain remnants from the formation of the planets.So, if you ever find yourself in a comet, make sure to pack your ice skates! Any other cosmic curiosities? What's


In [ ]:
trainer.model.push_to_hub("farziLLaMaTry1")

In [ ]:
MODEL_NAME = "FarziBuilder/farziHuggyFull"
folder_name = "checkSave1"
peft_config = PeftConfig.from_pretrained(folder_name)

# Provide the offload_dir path
offload_dir = "directory"  # replace with your actual directory path

# Check if the directory exists, if not create it
import os
if not os.path.exists(offload_dir):
    os.makedirs(offload_dir)

model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME, torch_dtype=torch.float16, load_in_8bit=False, device_map="auto", trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
folder_name = "checkSave1"
check_model = PeftModel.from_pretrained(model, folder_name)
check_model.eval()

print("PEFT model loaded successfully.")

# Merge LoRA and base model and save
print("Merging LoRA and base model...")
merged_model = check_model.merge_and_unload()

PEFT model loaded successfully.
Merging LoRA and base model...


In [ ]:
merged_model.save_pretrained("checkSave4")

In [ ]:
tokenizer.save_pretrained("checkSave4")

In [ ]:
merged_model.push_to_hub("farziLLaMatry2")
tokenizer.push_to_hub("farziLLaMatry2")